In [39]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [40]:
train = pd.read_csv('resources/mnist_train.csv')

In [41]:
train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 359.3 MB


In [81]:
np.random.seed(10)

remove_n = 45000
drop_indices = np.random.choice(train.index, remove_n, replace=False)
train_subset = train.drop(drop_indices)

In [82]:
train_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 2 to 59995
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 90.0 MB


In [83]:
train_subset.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
Y_train = train_subset["label"]
X_train = train_subset.drop(labels = ["label"],axis = 1) 

In [85]:
Y_train.value_counts()

1    1713
7    1550
8    1518
9    1515
3    1511
0    1504
4    1487
6    1472
2    1429
5    1301
Name: label, dtype: int64

In [86]:
X_train = X_train / 255.0

In [87]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [88]:
images_train, images_val, labels_train, labels_val = train_test_split(X_train, Y_train, test_size = 0.2, random_state=42)

In [89]:
images_train.shape

(12000, 784)

In [90]:
forest = RandomForestClassifier()

In [91]:
forest.fit(images_train, labels_train)

C:\Users\trug\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [95]:
class ModelEvaluator:

    def evaluate_model(self, forest, input, labels, limit):
        predictions = forest.predict(input)
        some_predictions = predictions[:limit]
        some_labels = list(labels)[:limit]

        differences = np.array(some_predictions) - np.array(some_labels)
        
        count_incorrectly_correctly_classified = []
        for difference in differences:
            if difference.any() != 0:
                count_incorrectly_correctly_classified.append(difference)
        print(len(count_incorrectly_correctly_classified))
        percentage_correctly_classified = (limit - len(count_incorrectly_correctly_classified)) / limit

        return some_predictions, percentage_correctly_classified

In [ ]:
evaluator = ModelEvaluator()
predicted, percentage_correct = evaluator.evaluate_model(forest, images_val, labels_val, 3000)
percentage_correct